# 🧠 Ultra Institutional AI Notebook – Updated

In [ ]:

# =========================================================
# 🧩 ULTRA INSTITUTIONAL AI – MASTER TOP-DOWN SIGNAL
# =========================================================
import numpy as np
import pandas as pd
from datetime import datetime

ultra_tf = ["1M","2M","3M","6M","1Y"]
higher_tf = ["1D","4H","1H"]
lower_tf  = ["5M","1M","30s"]
all_tfs = ultra_tf + higher_tf + lower_tf

def fetch_data(exchange, symbol, tf, limit=200):
    df = exchange.get_klines(symbol, tf, limit)
    return df

def add_institutional_indicators(df):
    df["ema_9"]  = df["close"].ewm(span=9).mean()
    df["ema_21"] = df["close"].ewm(span=21).mean()
    df["ema_50"] = df["close"].ewm(span=50).mean()
    delta = df["close"].diff()
    gain, loss = delta.clip(lower=0), -delta.clip(upper=0)
    df["rsi"] = 100 - (100 / (1 + gain.rolling(14).mean() / (loss.rolling(14).mean() + 1e-9)))
    df["macd"] = df["close"].ewm(span=12).mean() - df["close"].ewm(span=26).mean()
    df["macd_signal"] = df["macd"].ewm(span=9).mean()
    tr = pd.concat([df["high"]-df["low"], abs(df["high"]-df["close"].shift()), abs(df["low"]-df["close"].shift())], axis=1).max(axis=1)
    df["atr"] = tr.rolling(14).mean()
    sma20 = df["close"].rolling(20).mean()
    std20 = df["close"].rolling(20).std()
    df["bb_upper"] = sma20 + 2*std20
    df["bb_lower"] = sma20 - 2*std20
    hl2 = (df["high"] + df["low"])/2
    factor = 3
    df["supertrend_upper"] = hl2 + factor*df["atr"]
    df["supertrend_lower"] = hl2 - factor*df["atr"]
    df['hammer'] = ((df['close'] > df['open']) & ((df['low'] - df[['open','close']].min(axis=1)) > 2*(df['close']-df['open']))).astype(int)
    df['doji'] = (abs(df['close'] - df['open']) <= 0.1*(df['high'] - df['low'])).astype(int)
    df['engulfing_bull'] = ((df['close'] > df['open'].shift()) & (df['open'] < df['close'].shift())).astype(int)
    df['momentum_score'] = df['hammer'] + df['doji'] + df['engulfing_bull']
    df.fillna(method="bfill", inplace=True)
    df.fillna(method="ffill", inplace=True)
    return df

def evaluate_strategies(df):
    scores = {}
    scores['breakout'] = 0.85 if df['macd'].iloc[-1] > df['macd_signal'].iloc[-1] else 0.7
    scores['mean_reversion'] = 0.8 if df['rsi'].iloc[-1] < 30 else 0.6
    scores['momentum_reversal'] = df['momentum_score'].iloc[-1]/3
    return scores

def evaluate_indicators(df):
    scores = {}
    scores['ema_align'] = 0.9 if df['ema_9'].iloc[-1] > df['ema_21'].iloc[-1] else 0.7
    scores['atr_spike'] = 0.85 if df['atr'].iloc[-1] > df['atr'].rolling(14).mean().iloc[-1]*1.5 else 0.6
    return scores

def news_sentiment_score(symbol):
    return 0.85

def top_down_alignment(exchange, symbol):
    tf_results = {}
    for tf in all_tfs:
        df = fetch_data(exchange, symbol, tf)
        if df.empty: continue
        df = add_institutional_indicators(df)
        strat_scores = evaluate_strategies(df)
        ind_scores = evaluate_indicators(df)
        combined_scores = {**{k:v for k,v in strat_scores.items() if v>=0.8},
                           **{k:v for k,v in ind_scores.items() if v>=0.8}}
        tf_results[tf] = combined_scores
    selected_tfs = sorted(tf_results, key=lambda x: np.mean(list(tf_results[x].values() or [0])), reverse=True)[:5]
    final_alignment = {}
    for tf in selected_tfs:
        final_alignment.update(tf_results[tf])
    final_alignment['news_sentiment'] = news_sentiment_score(symbol)
    return final_alignment, selected_tfs

def execute_trade(exchange, symbol, alignment, selected_tfs):
    leverage = 10 if selected_tfs[0] in ultra_tf + higher_tf else 5
    df_low = fetch_data(exchange, symbol, lower_tf[0])
    entry_price = df_low['close'].iloc[-1]
    df_low = add_institutional_indicators(df_low)
    atr = df_low['atr'].iloc[-1]
    tp = entry_price + atr*1.5
    sl = entry_price - atr*1.5
    trade_decision = {
        "symbol": symbol,
        "entry": entry_price,
        "tp": tp,
        "sl": sl,
        "leverage": leverage,
        "alignment": alignment,
        "selected_tfs": selected_tfs,
        "timestamp": datetime.utcnow().isoformat()
    }
    return trade_decision

# Example (commented):
# alignment, selected_tfs = top_down_alignment(exchange, "BTCUSDT")
# trade_decision = execute_trade(exchange, "BTCUSDT", alignment, selected_tfs)
# print("Trade decision:", trade_decision)


## 🔁 Candle Learning Module (CLM) — Integrated
This cell contains the candle pattern detector, reinforcement update logic, and helper functions to integrate candle confidence into the master top-down alignment. The CLM stores pattern performance to `/mnt/data/candle_stats.json`.

In [ ]:

import json
import os
import numpy as np
import pandas as pd
from collections import defaultdict

# Path to store pattern stats
CANDLE_STATS_PATH = "/mnt/data/candle_stats.json"

# Initialize stats file if missing
def _ensure_stats_file():
    if not os.path.exists(CANDLE_STATS_PATH):
        init = {}
        # initialize common patterns with neutral stats
        patterns = ["bullish_engulfing","bearish_engulfing","hammer","hanging_man","pinbar","doji","marubozu","morning_star","evening_star","tweezers_top","tweezers_bottom"]
        for p in patterns:
            init[p] = {"success_rate": 0.5, "count": 0, "wins": 0, "losses": 0}
        with open(CANDLE_STATS_PATH, "w") as f:
            json.dump(init, f, indent=2)

_ensure_stats_file()

def load_candle_stats():
    with open(CANDLE_STATS_PATH, "r") as f:
        return json.load(f)

def save_candle_stats(stats):
    with open(CANDLE_STATS_PATH, "w") as f:
        json.dump(stats, f, indent=2)

# --- Pattern detection routines ---
def detect_patterns_from_df(df):
    \"\"\"Detect common candle patterns on the last 3 bars and return pattern signals and raw indicators.\"\"\"
    # Ensure required cols exist
    for col in ["open","high","low","close","volume"]:
        if col not in df.columns:
            raise ValueError(f\"Missing required column: {col}\")
    last = df.iloc[-3:].copy().reset_index(drop=True)  # work with last 3 for multi-bar patterns
    patterns_found = defaultdict(int)
    # Single bar metrics
    last['body'] = (last['close'] - last['open']).abs()
    last['upper_wick'] = last['high'] - last[['close','open']].max(axis=1)
    last['lower_wick'] = last[['close','open']].min(axis=1) - last['low']
    last['range'] = last['high'] - last['low'] + 1e-9
    last['body_ratio'] = last['body'] / last['range']

    # Detect doji (last bar)
    if last.loc[2,'body_ratio'] <= 0.1:
        patterns_found['doji'] += 1

    # Detect hammer / hanging man (last bar)
    if last.loc[2,'body_ratio'] < 0.4 and last.loc[2,'lower_wick'] > 2 * last.loc[2,'body']:
        # bullish if close > open
        if last.loc[2,'close'] > last.loc[2,'open']:
            patterns_found['hammer'] += 1
        else:
            patterns_found['hanging_man'] += 1

    # Engulfing (2-bar)
    if (last.loc[1,'close'] < last.loc[1,'open']) and (last.loc[2,'close'] > last.loc[2,'open']) and (last.loc[2,'open'] < last.loc[1,'close']) and (last.loc[2,'close'] > last.loc[1,'open']):
        patterns_found['bullish_engulfing'] += 1
    if (last.loc[1,'close'] > last.loc[1,'open']) and (last.loc[2,'close'] < last.loc[2,'open']) and (last.loc[2,'open'] > last.loc[1,'close']) and (last.loc[2,'close'] < last.loc[1,'open']):
        patterns_found['bearish_engulfing'] += 1

    # Pinbar (long upper or lower wick relative to body)
    if last.loc[2,'upper_wick'] > 3 * last.loc[2,'body']:
        patterns_found['pinbar_upper'] += 1
    if last.loc[2,'lower_wick'] > 3 * last.loc[2,'body']:
        patterns_found['pinbar_lower'] += 1

    # Marubozu (very small wicks)
    if last.loc[2,'upper_wick'] < 0.05*last.loc[2,'range'] and last.loc[2,'lower_wick'] < 0.05*last.loc[2,'range'] and last.loc[2,'body_ratio']>0.8:
        patterns_found['marubozu'] += 1

    # Tweezer bottom/top (2-bar)
    if abs(last.loc[1,'low'] - last.loc[2,'low'])/max(1e-9,last.loc[1,'low']) < 0.002:
        patterns_found['tweezers_bottom'] += 1
    if abs(last.loc[1,'high'] - last.loc[2,'high'])/max(1e-9,last.loc[1,'high']) < 0.002:
        patterns_found['tweezers_top'] += 1

    return dict(patterns_found)

# --- Candle confidence scoring ---
def candle_confidence_from_patterns(patterns):
    \"\"\"Compute a 0-1 confidence score using stored stats and detected patterns.\"\"\"
    stats = load_candle_stats()
    # base score
    score = 0.5
    weight_sum = 0.0
    for p, count in patterns.items():
        if p not in stats:
            continue
        pat_stat = stats[p]
        pat_score = pat_stat.get('success_rate', 0.5)
        # weight patterns by occurrences and their historical reliability
        w = min(1.0, count)  # occurrence weight (cap 1)
        score += pat_score * w
        weight_sum += w
    if weight_sum > 0:
        score = score / (1 + weight_sum)  # normalize into 0-1-ish range
    # clip
    return float(max(0.0, min(1.0, score)))

# --- Reinforcement update ---
def record_pattern_outcomes(patterns, outcome):
    \"\"\"Update pattern stats : outcome True=win, False=loss\"\"\"
    stats = load_candle_stats()
    for p, cnt in patterns.items():
        if p not in stats:
            stats[p] = {"success_rate":0.5, "count":0, "wins":0, "losses":0}
        stats[p]['count'] += cnt
        if outcome:
            stats[p]['wins'] += cnt
        else:
            stats[p]['losses'] += cnt
        # recompute success_rate with smoothing
        wins = stats[p]['wins']
        losses = stats[p]['losses']
        total = wins + losses
        # Bayesian smoothing with prior 0.5, prior_weight=3
        prior = 0.5; prior_w = 3
        stats[p]['success_rate'] = (wins + prior*prior_w) / (total + prior_w) if total>0 else stats[p]['success_rate']
    save_candle_stats(stats)
    return stats

# --- Helper integrate function ---
def get_candle_confidence_for_tf(exchange, symbol, tf):
    df = fetch_data(exchange, symbol, tf)
    if df.empty:
        return 0.5, {}
    patterns = detect_patterns_from_df(df)
    conf = candle_confidence_from_patterns(patterns)
    return conf, patterns

# --- Example usage ---
# conf, patterns = get_candle_confidence_for_tf(exchange, 'BTCUSDT', '5M')
# print(conf, patterns)

# --- Utility to record a trade outcome after execution ---
def record_trade_result(symbol, tf, patterns, entry_price, exit_price):
    \"\"\"Call this after a trade is closed to update CLM learning stats.\"\"\"
    win = (exit_price - entry_price) > 0
    stats = record_pattern_outcomes(patterns, win)
    return stats



## 🔁 Retrain Command (manual trigger)
This cell adds a `/retrain`-style function you can call from the notebook to update **candle pattern stats**, **strategy weights**, and **timeframe alignment weights** based on historical performance stored in `/mnt/data/candle_stats.json` and recent trade logs.

How to use on mobile:
1. Open this notebook in Jupyter (or Colab).  
2. Run the **Retrain** code cell below (it will run once, update weights, and save results).  
3. Output and summary will print in the cell output.  
4. Files created: `/mnt/data/retrain_log.json` (summary) and `/mnt/data/strategy_timeframe_weights.json` (updated weights).


In [ ]:

import json, os, math, statistics, datetime
from collections import defaultdict

CANDLE_STATS_PATH = "/mnt/data/candle_stats.json"
RETRAIN_LOG_PATH = "/mnt/data/retrain_log.json"
WEIGHTS_PATH = "/mnt/data/strategy_timeframe_weights.json"
TRADE_HISTORY_PATH = "/mnt/data/trade_history.json"  # optional: used if exists

def _safe_load(path, default):
    if os.path.exists(path):
        with open(path,"r") as f:
            return json.load(f)
    return default

def _safe_save(path, obj):
    with open(path,"w") as f:
        json.dump(obj, f, indent=2)

def retrain_all(update_timeframe_weights=True, update_strategy_weights=True, smoothing=3):
    \"\"\"Run a retraining pass:
    - Reads candle pattern stats
    - Updates smoothed success rates
    - Generates/updates strategy and timeframe weights based on recent outcomes (trade_history if available)
    \"\"\"
    now = datetime.datetime.utcnow().isoformat()
    stats = _safe_load(CANDLE_STATS_PATH, {})
    if not stats:
        print("⚠️ No candle_stats found at", CANDLE_STATS_PATH)
        return None

    # 1) Recompute smoothed success rates (already updated by CLM during record_trade_result)
    # Apply an additional smoothing to ensure stability
    summary = {"timestamp": now, "patterns": {}, "strategy_weights": {}, "timeframe_weights": {}}

    # Update pattern stats (apply small smoothing toward 0.5 prior)
    for p, v in stats.items():
        wins = v.get("wins",0)
        losses = v.get("losses",0)
        total = wins + losses
        # Bayesian smoothing
        prior = 0.5; prior_w = smoothing
        success_rate = (wins + prior*prior_w) / (total + prior_w) if total>0 else v.get("success_rate",0.5)
        # store back
        stats[p]['success_rate'] = round(success_rate,4)
        summary["patterns"][p] = {"success_rate": stats[p]['success_rate'], "count": stats[p].get("count",0)}
    _safe_save(CANDLE_STATS_PATH, stats)

    # 2) Strategy & timeframe weight updates (heuristic if trade history exists)
    # Default strategies and timeframes (will extend if trade_history contains more)
    default_strategies = ["breakout","mean_reversion","momentum_reversal"]
    default_timeframes = ["30s","5M","1M","1H","4H","1D","1W","1M","3M","6M","1Y"]

    # initialize weights
    strategy_weights = {s: 1.0 for s in default_strategies}
    timeframe_weights = {tf: 1.0 for tf in default_timeframes}

    # If trade history exists, use it to adjust weights
    trade_history = _safe_load(TRADE_HISTORY_PATH, [])
    if trade_history:
        # Expected trade_history entry: {"symbol","tf","strategy","entry","exit","win":bool, "timestamp"}
        strat_perf = defaultdict(lambda: {"wins":0,"losses":0})
        tf_perf = defaultdict(lambda: {"wins":0,"losses":0})
        for t in trade_history:
            s = t.get("strategy")
            tf = t.get("tf")
            win = bool(t.get("win"))
            if s:
                if win: strat_perf[s]["wins"] += 1
                else: strat_perf[s]["losses"] += 1
            if tf:
                if win: tf_perf[tf]["wins"] += 1
                else: tf_perf[tf]["losses"] += 1

        # Convert to weighted scores
        for s, perf in strat_perf.items():
            w = (perf["wins"] + smoothing*0.5) / (perf["wins"]+perf["losses"]+smoothing)
            strategy_weights[s] = round(w,4)
        for tf, perf in tf_perf.items():
            w = (perf["wins"] + smoothing*0.5) / (perf["wins"]+perf["losses"]+smoothing)
            timeframe_weights[tf] = round(w,4)

    # Normalize weights to 0-1
    max_sw = max(strategy_weights.values()) if strategy_weights else 1.0
    strategy_weights = {k: round(v/max_sw,4) for k,v in strategy_weights.items()}

    max_tw = max(timeframe_weights.values()) if timeframe_weights else 1.0
    timeframe_weights = {k: round(v/max_tw,4) for k,v in timeframe_weights.items()}

    # Save weights
    weights = {"strategy_weights": strategy_weights, "timeframe_weights": timeframe_weights, "generated_at": now}
    _safe_save(WEIGHTS_PATH, weights)
    summary["strategy_weights"] = strategy_weights
    summary["timeframe_weights"] = timeframe_weights

    # 3) Log retrain summary
    _safe_save(RETRAIN_LOG_PATH, summary)
    print("✅ Retrain complete. Summary saved to", RETRAIN_LOG_PATH)
    return summary

# Helper to call from other code (mimics /retrain)
def handle_retrain_command():
    print("Running retrain...")
    summary = retrain_all()
    if summary:
        print("Retrain summary:")
        print(json.dumps(summary, indent=2))
    return summary

# Expose as global for notebook usage
_retrain_available = True
print("Retrain utility loaded. Call handle_retrain_command() to retrain (manual trigger).")
